<a href="https://colab.research.google.com/github/Kruthika2811/Tuberculosis-Early-Prediction/blob/main/FinalYearProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import numpy as np
# import pandas as pd
# import os
# print(os.getcwd())

# df = pd.read_csv("tuberculosis_classification_with_ammonia.csv")

# # Fever → Temperature in Celsius
# df["Temperature"] = df["Fever"].apply(lambda x: np.random.uniform(38,40) if x=="Yes" else np.random.uniform(36,37.2))

# # Cough → keep as binary (for now)
# df["Cough_Binary"] = df["Cough"].apply(lambda x: 1 if x=="Yes" else 0)

# # Night Sweats & Weight Loss → keep as categorical or drop

# # Ammonia Level → scale to ppm range for MQ-135 (say 50–400 ppm)
# min_val, max_val = df["Ammonia Level"].min(), df["Ammonia Level"].max()
# df["MQ135_ppm"] = 50 + (df["Ammonia Level"] - min_val) / (max_val - min_val) * (400-50)

# # Add simulated heart rate
# df["HeartRate"] = df["TB"].apply(lambda x: np.random.randint(90,120) if x=="Yes" else np.random.randint(60,90))


/content


FileNotFoundError: [Errno 2] No such file or directory: 'tuberculosis_classification_with_ammonia.csv'

In [1]:
from google.colab import files
uploaded = files.upload()


Saving tb_major_clinical.csv to tb_major_clinical.csv


In [ ]:
# import numpy as np
# import pandas as pd
# import os
# print(os.getcwd())

# df = pd.read_csv("tuberculosis_classification_with_ammonia.csv")

# # Fever → Temperature in Celsius
# df["Temperature"] = df["Fever"].apply(lambda x: np.random.uniform(38,40) if x=="Yes" else np.random.uniform(36,37.2))

# # Cough → keep as binary (for now)
# df["Cough_Binary"] = df["Cough"].apply(lambda x: 1 if x=="Yes" else 0)

# # Night Sweats & Weight Loss → keep as categorical or drop

# # Ammonia Level → scale to ppm range for MQ-135 (say 50–400 ppm)
# min_val, max_val = df["AmmoniaRate"].min(), df["AmmoniaRate"].max()
# df["MQ135_ppm"] = 50 + (df["AmmoniaRate"] - min_val) / (max_val - min_val) * (400-50)

# # Add simulated heart rate
# df["HeartRate"] = df["ChestPain"].apply(lambda x: np.random.randint(90,120) if x=="Yes" else np.random.randint(60,90))

# # Save the modified DataFrame to a new CSV
# df.to_csv("tuberculosis_with_features.csv", index=False)


/content


In [ ]:
# print(df.columns)


Index(['PatientID', 'Age', 'Gender', 'Cough', 'Fever', 'WeightLoss',
       'NightSweats', 'ChestPain', 'Hemoptysis', 'Breathlessness',
       'ContactHistory', 'TravelHistory', 'HIVStatus', 'PreviousTB',
       'ChestXRay', 'SputumTest', 'Diagnosis', 'AmmoniaRate', 'Temperature',
       'Cough_Binary'],
      dtype='object')


In [2]:
!pip install imbalanced-learn


In [3]:
# 📌 TB Dataset Preprocessing Code
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE   # for imbalance handling

# 1. Load dataset
df = pd.read_csv("tb_major_clinical.csv")

# 2. Encode categorical variable (sex)
le = LabelEncoder()
df["sex"] = le.fit_transform(df["sex"])   # Male=0, Female=1

# 3. Separate features (X) and target (y)
X = df.drop("tb_positive", axis=1)
y = df["tb_positive"]

# 4. Scale numerical features
scaler = StandardScaler()
num_features = ["age", "heart_rate_bpm", "temperature_c", "breath_ammonia_ppm", "bmi"]

X[num_features] = scaler.fit_transform(X[num_features])

# 5. Handle imbalance (if needed)
print("Class distribution before SMOTE:\n", y.value_counts())

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("Class distribution after SMOTE:\n", y_resampled.value_counts())

# 6. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


Class distribution before SMOTE:
 tb_positive
0    1594
1     406
Name: count, dtype: int64
Class distribution after SMOTE:
 tb_positive
0    1594
1    1594
Name: count, dtype: int64
Train shape: (2550, 15)
Test shape: (638, 15)


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. Train the model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# 2. Make predictions
y_pred = model.predict(X_test)

# 3. Evaluate performance
print("✅ Model Evaluation")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


✅ Model Evaluation
Accuracy: 0.9639498432601881

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.97      0.96       319
           1       0.97      0.96      0.96       319

    accuracy                           0.96       638
   macro avg       0.96      0.96      0.96       638
weighted avg       0.96      0.96      0.96       638


Confusion Matrix:
 [[308  11]
 [ 12 307]]


In [5]:
import joblib

# Save model
joblib.dump(model, "tb_model.pkl")

# Save preprocessing objects
joblib.dump(le, "label_encoder_sex.pkl")
joblib.dump(scaler, "scaler.pkl")

print("✅ Model, encoder, and scaler saved as .pkl files")


✅ Model, encoder, and scaler saved as .pkl files
